# Pré-processamento de dados de dados

In [8]:
from datetime import datetime

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, normalize
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix, vstack

interaction_columns = [
    'pageVisitsCountHistory',
    'scrollPercentageHistory',
]



def calc_recency_score(issued: pd.Series):
    """
    Calcula o score de recência para uma série de timestamps.

    O score é inversamente proporcional ao tempo decorrido,
    dando mais prioridade a notícias mais recentes.

    :param issued: Série com os timestamps das notícias.
    :return: Série com os scores de recência.
    """
    # Converter os timestamps para milissegundos

    now = parse_date_timestamp(datetime.timestamp(datetime.now()))

    # Calcular o score de recência, evitando divisão por zero
    return 1 / np.maximum(now - parse_date_timestamp(issued), 1e-6)


def parse_date_timestamp(date):
    # Converte para datetime se ainda for string
    if isinstance(date, str):
        date = pd.to_datetime(date, errors='coerce')
    elif isinstance(date, float):
        return (date * 1000)

    return int(datetime.timestamp(date) * 1000)


def build_interaction_matrix(df, interaction_columns, chunk_size=10000):
    """
    Constrói uma matriz de interação a partir de um DataFrame específico em chunks.
    Garante que todas as matrizes parciais tenham as mesmas colunas automaticamente.
    Retorna uma matriz esparsa empilhada verticalmente.
    """
    interaction_matrix = []

    # Extrai todas as colunas únicas automaticamente do DataFrame
    all_columns = df['history'].unique()
    print(f"Colunas únicas encontradas: {len(all_columns)}")

    for start in tqdm(range(0, len(df), chunk_size), desc='Building interaction matrix'):
        chunk = df.iloc[start:start + chunk_size]

        # Gera a matriz de interação parcial
        partial_matrix = chunk.pivot_table(
            index='userId',
            columns='history',
            values=interaction_columns,
            aggfunc='mean',
            fill_value=0
        )

        # Reindexa para garantir a consistência das colunas
        partial_matrix = partial_matrix.reindex(columns=all_columns, fill_value=0)

        # Converte para matriz esparsa e acumula
        interaction_matrix.append(csr_matrix(partial_matrix))

    # Retorna a matriz esparsa empilhada verticalmente
    return vstack(interaction_matrix)




def transform_user_to_info_user_item(user_info):
    user_item = user_info[['userId', 'history', 'timestampHistory', 'numberOfClicksHistory', 'timeOnPageHistory', 'scrollPercentageHistory', 'pageVisitsCountHistory', 'timestampHistory_new']]

    user_item = user_item.set_index('userId').apply(lambda row: row.str.split(','), axis=1)
    user_item = user_item.apply(pd.Series.explode).reset_index()
    user_item['numberOfClicksHistory'] = user_item['numberOfClicksHistory'].astype(int)
    user_item['timeOnPageHistory'] = user_item['timeOnPageHistory'].astype(int)
    user_item['scrollPercentageHistory'] = user_item['scrollPercentageHistory'].astype(float)
    user_item['timestampHistory'] = pd.to_numeric(user_item['timestampHistory'], errors='coerce').fillna(0).astype(int)
    user_item['pageVisitsCountHistory'] = pd.to_numeric(user_item['pageVisitsCountHistory'], errors='coerce').fillna(0).astype(int)
    user_item['history'] = user_item['history'].str.strip()
    user_item['userId'] = user_item['userId'].str.strip()
    return user_item


In [10]:
# 🆔 Obter o índice numérico do usuário

import pandas as pd
import glob
from scipy.sparse import csr_matrix
from sklearn.preprocessing import StandardScaler

# Lista para acumular as matrizes parciais
interaction_matrices = []

user_infos = []

for fpath in glob.glob('../data/raw/files/treino/*.csv'):
    print(f"Processando: {fpath}")
    df = pd.read_csv(fpath)
    df = transform_user_to_info_user_item(df)

    interaction_matrices.append(build_interaction_matrix(df, interaction_columns))
    user_infos.append(df)

Processando: ../data/raw/files/treino/treino_parte6.csv
Colunas únicas encontradas: 94177


Building interaction matrix: 100%|██████████| 112/112 [01:06<00:00,  1.67it/s]


Processando: ../data/raw/files/treino/treino_parte3.csv
Colunas únicas encontradas: 107743


Building interaction matrix: 100%|██████████| 142/142 [01:36<00:00,  1.48it/s]


Processando: ../data/raw/files/treino/treino_parte4.csv
Colunas únicas encontradas: 107578


...

Building interaction matrix: 100%|██████████| 138/138 [01:34<00:00,  1.46it/s]


Processando: ../data/raw/files/treino/treino_parte1.csv
Colunas únicas encontradas: 108573


Building interaction matrix: 100%|██████████| 143/143 [01:33<00:00,  1.53it/s]


Processando: ../data/raw/files/treino/treino_parte2.csv
Colunas únicas encontradas: 106992


Building interaction matrix: 100%|██████████| 140/140 [01:35<00:00,  1.46it/s]


In [18]:
from scipy.sparse import vstack
from sklearn.preprocessing import StandardScaler
import pandas as pd

user_info = pd.concat(user_infos, ignore_index=True)

# Define um conjunto fixo de colunas baseado no DataFrame completo
all_columns = user_infos['history'].unique()
all_columns = pd.Index(all_columns)

# Concatena todas as matrizes esparsas em uma matriz final
if interaction_matrices:
    print("Concatenando matrizes...")
    interaction_matrix = vstack(interaction_matrices)

    # Normaliza e converte para matriz esparsa
    print("Normalizando e convertendo para matriz esparsa...")
    interaction_matrix = StandardScaler().fit_transform(interaction_matrix.toarray())
    interaction_matrix = csr_matrix(interaction_matrix)

    # Extrai categorias de usuários e itens
    user_id_category = pd.Categorical(range(interaction_matrix.shape[0]))
    history_id_category = pd.Categorical(range(interaction_matrix.shape[1]))

    print("Matriz de interação criada com sucesso!")
else:
    print("Nenhum dado processado!")

Concatenando matrizes...


ValueError: incompatible dimensions for axis 1

In [5]:
news_item = pd.concat([pd.read_csv(fpath) for fpath in glob.glob('../data/raw/itens/itens/*.csv')])
news_item.head()

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém\nÁdrio Denner/ AD Prod...,As expressões santarenas não significam apenas...
2,61e07f64-cddf-46f2-b50c-ea0a39c22050,http://g1.globo.com/mundo/noticia/2022/07/08/e...,2022-07-08 08:55:52+00:00,2023-04-15 04:25:39+00:00,Ex-premiê Shinzo Abe morre após ser baleado no...,Novo vídeo mostra que assassino de Shinzo Abe ...,Ex-primeiro-ministro foi atingido por tiros de...
3,30e2e6c5-554a-48ed-a35f-6c6691c8ac9b,http://g1.globo.com/politica/noticia/2021/09/0...,2021-09-09 19:06:46+00:00,2023-06-07 17:44:54+00:00,"Relator no STF, Fachin vota contra marco tempo...","Relator no STF, Fachin vota contra marco tempo...",Ministro defendeu que posse indígena é diferen...
4,9dff71eb-b681-40c7-ac8d-68017ac36675,http://g1.globo.com/politica/noticia/2021/09/1...,2021-09-15 19:16:13+00:00,2023-06-07 17:43:39+00:00,"\nApós 2 votos, pedido de vista suspende julga...",Após um pedido de vista (mais tempo para análi...,"Pelo marco temporal, índios só podem reivindic..."


## Criando score de recencia e score de populariadade por noticia

In [16]:
scaler = MinMaxScaler()


count_visits_by_news = (user_infos
                        .groupby('history')['pageVisitsCountHistory']
                        .count())
news_item['issued'] = pd.to_datetime(news_item['issued'], errors='coerce')
news_item['recency_score'] = scaler.fit_transform(
    news_item['issued'].apply(calc_recency_score).fillna(0).values.reshape(-1, 1)
)


news_item = (news_item.set_index('page').join(count_visits_by_news, how='inner').reset_index())
news_item.rename(columns={'pageVisitsCountHistory': 'popularity_score'}, inplace=True)
news_item.rename(columns={'index': 'history'},inplace=True)
news_item.head()

KeyError: "None of ['page'] are in the columns"

# Treinando o modelo

In [18]:
from sklearn.decomposition import TruncatedSVD


def apply_svd(interaction_matrix, n_components=50):
    # Fatoração de matriz esparsa com SVD truncado
    svd = TruncatedSVD(n_components=n_components)
    user_latent_matrix = svd.fit_transform(interaction_matrix)
    item_latent_matrix = svd.components_.T  # Vetores latentes dos itens

    # Normalizar os vetores latentes
    user_latent_matrix = normalize(user_latent_matrix)
    item_latent_matrix = normalize(item_latent_matrix)

    return user_latent_matrix, item_latent_matrix, svd

# Criando a função de recomendação

In [19]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def recommend_for_user(user_id, user_category, item_category, user_latent_matrix, item_latent_matrix, top_k=10):
    # Obter o índice do usuário e o vetor latente correspondente
    user_index = user_category.cat.categories.get_loc(user_id)

    user_vector = user_latent_matrix[user_index].reshape(1, -1)

    # Calcular a similaridade entre o usuário e todos os itens
    similarities = cosine_similarity(user_vector, item_latent_matrix).flatten()

    # Obter os índices dos itens mais similares
    top_indices = np.argsort(-similarities)[:top_k]

    recommended_items = item_category.cat.categories[top_indices]

    return recommended_items

In [ ]:
user_latent_matrix, item_latent_matrix,_ = apply_svd(interaction_matrix, n_components=50)

In [229]:
user_id = 'ec1639851d99586c7f4da928deb49187303aec6e3b8d66c0359d4920e3c105e6'  # Exemplo de ID de usuário

recomendacoes = recommend_for_user(user_id, user_id_category, history_id_category, user_latent_matrix, interaction_matrix, top_k=10)


filtros = [
    'b7b90e18-7613-4ca0-a8fc-fd69addfcd85',
    '835fdd8c-30e6-46f7-9958-9ac8867bdf05',
    'dbe79e5d-6364-4efa-bba0-d22c880a073a',
    'bfc8ed3d-1cac-4f73-a978-e42e8013b325',
    'a433fd47-ed48-4f96-ba2a-961bd99a301b',
    '9971bdc4-7a95-45e4-a087-83ff44baa7f5',#Ignorar esse, ele é mais para teste
    '9c9a6c75-be7b-49dc-9c22-f6a76e46c3df' #Ignorar esse, ele é mais para teste
]

news_item.set_index('history').loc[recomendacoes].query("index in @filtros")



,url,issued,modified,title,body,caption,recency_score,popularity_score


In [227]:
user_infos.query("userId == 'ec1639851d99586c7f4da928deb49187303aec6e3b8d66c0359d4920e3c105e6' and history in @filtros")

,userId,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new,interaction_score
6406021,ec1639851d99586c7f4da928deb49187303aec6e3b8d66...,9971bdc4-7a95-45e4-a087-83ff44baa7f5,1657080433285,0,59536,60.41,1,1657080433285,11913.541


# Salvando dados no datawarehorse

In [228]:
from sqlalchemy import create_engine

# 💾 Salvar no SQLite
engine = create_engine('sqlite:///../data/refined/datawarehouse.db', echo=False)

# Salvar os DataFrames no banco de dados
user_infos.to_sql('merged_data', con=engine, if_exists='replace', index=False)
news_item.to_sql('news_item', con=engine, if_exists='replace', index=False)

PendingRollbackError: Can't reconnect until invalid transaction is rolled back.  Please rollback() fully before proceeding (Background on this error at: https://sqlalche.me/e/20/8s2b)